In [ ]:
### 以下为，从初始提报开始，对八月项目业绩进行核算的完整代码
# 其中，需要滚动8月明细的提报包括：7月非月度项目+8月全部项目
# 如核算9月，则用7.8月非月度项目+9月全部项目

In [ ]:
import pandas as pd
import numpy as np
import os

path = 'D:\\maran10\\Documents\\日常工作\\3.项目业绩\\商品池\\'
path_oldreport = path+'8月\\提报处理\\'
path_newreport = path+'8月\\提报\\'
path_resave = path+'8月\\提报处理\\'
path_save = path+'8月\\项目业绩\\'

In [ ]:
# 提报数据处理 
# 由于提报数据为手工填写，注意要根据数据类型不同做相应的格式转换改动
# 提报有两种，一种是按“主数据号”进行提报的，以下命名为pro_T
# 另一种是没有主数据号，按“前台账号”进行提报的，以下命名为pro_F

def standard(a):
    spl=[a]
    if "," in a:
        spl = a.replace(',', '\n').split('\n')
    if "，" in a:
        spl = a.replace('，', '\n').split('\n')
    if "\n" in a:
        spl = a.split("\n")
    return spl    


def expand(x):
    data = {c: x[c].iloc[0] for c in x}
    data['*主数据号\n（与前台账号二选一，一行只能填一个）'] = standard(data['*主数据号\n（与前台账号二选一，一行只能填一个）'])
    if len(data['*主数据号\n（与前台账号二选一，一行只能填一个）'])==1:
        return pd.DataFrame(data, index=[0])
    return pd.DataFrame(data)

count=0
new_reports = os.listdir(path_newreport)
for f in new_reports:
    if not ".xlsx" in f:
        continue
    pro = pd.read_excel(path_newreport+f) 
    pro = pro[pro['是否为项目SKU']=='是']
    pro['提报月份'] = ['8月']*len(pro)
    print(f,len(pro))
    
    pro_F = pro[pd.isnull(pro['*主数据号\n（与前台账号二选一，一行只能填一个）'])].copy()
    print(f,'主数据号为空的',len(pro_F))

    pro_F['*SKU商品编码'] = pro_F['*SKU商品编码'].apply(lambda x:int(x))
    pro_F['*SKU商品编码'] = pro_F['*SKU商品编码'].apply(lambda x:str(x))

    pro_F['*SKU商品编码'] = pro_F['*SKU商品编码'].apply(lambda x:x.strip())
    pro_F['*前台账号\n（与主数据号二选一，一行只能填一个）'] = pro_F['*前台账号\n（与主数据号二选一，一行只能填一个）'].apply(lambda x:x.strip())
    
    pro_F['*区域']=pro_F['*区域'].apply(lambda x: x[0:2])
    
    
    pro_T = pro[pd.notnull(pro['*主数据号\n（与前台账号二选一，一行只能填一个）'])].copy()
    print(f,"主数据号不为空的",len(pro_T))
    pro_T['序号'] = np.arange(len(pro_T))
    proT_expand = pro_T.groupby('序号').apply(expand)
    proT_expand.index = np.arange(len(proT_expand))
    

    proT_expand['*主数据号\n（与前台账号二选一，一行只能填一个）'] = proT_expand['*主数据号\n（与前台账号二选一，一行只能填一个）'].apply(lambda x:str(x))

    proT_expand['*主数据号\n（与前台账号二选一，一行只能填一个）'] = proT_expand['*主数据号\n（与前台账号二选一，一行只能填一个）'].apply(lambda x:x.strip())
    print(f,"标准化主数据号后",len(proT_expand))
    
    proT_expand = proT_expand.drop(columns = ['序号'])
    
    proT_expand['*主数据号\n（与前台账号二选一，一行只能填一个）'] = proT_expand['*主数据号\n（与前台账号二选一，一行只能填一个）'].apply(lambda x:x.strip())
    
    proT_expand['*SKU商品编码'] = proT_expand['*SKU商品编码'].apply(lambda x:int(x))
    proT_expand['*SKU商品编码'] = proT_expand['*SKU商品编码'].apply(lambda x: str(x))
    proT_expand['*SKU商品编码'] = proT_expand['*SKU商品编码'].apply(lambda x:x.strip())
    
    proT_expand['*区域']=proT_expand['*区域'].apply(lambda x:str(x))
    proT_expand['*区域']=proT_expand['*区域'].apply(lambda x: x[0:2])
    proT_expand['id'] = proT_expand['*主数据号\n（与前台账号二选一，一行只能填一个）']+','+proT_expand['*SKU商品编码']+','+proT_expand['*区域']+'区'
    print(f,"生成id后",len(proT_expand))
    
    
    if count == 0:
        report1_total = pd.DataFrame(columns = list(proT_expand))
        report2_total = pd.DataFrame(columns = list(pro_F))
    report1_total = pd.concat([report1_total,proT_expand])
    report2_total = pd.concat([report2_total,pro_F])
    count = count+1
    print(count,len(report1_total),len(report2_total))


report1_total.to_excel(path_resave+'8月商品池提报汇总_主数据号不为空.xlsx',index=False,encoding='gbk')
report2_total.to_excel(path_resave+'8月商品池提报汇总_主数据号为空.xlsx',index=False,encoding='gbk')

# 要记得检查 report1_total 中 id部分“,xxx,xxx”的，这些是提报时候多打了空行的，需要手动删除
# 对 report2_total ,去黄金眼中查询，有主数据号的要补上主数据号，按主数据号的方式生成id，再贴回主数据号不为空（report1_total）的明细中
# 对于没有主数据号的，用前台账号代替主数据号生成id后，补入 report1_total
# 补好的文件，以下命名为“x月商品池提报汇总_汇总后”

In [ ]:
report_new = pd.read_excel(path_resave+'8月商品池提报汇总_汇总后.xlsx',encoding='gbk')
report_new['项目类型'] = report_new['项目类型'].apply(lambda x:x[0:2])

# 下面要将7月的提报读入，汇总7.8月的全部提报
report_old = pd.read_excel(path_oldreport+'截至8月中商品池提报汇总_9.11_以此为准.xlsx',encoding='gbk')
report_total = pd.concat([report_old,report_new])
print('提报汇总',len(report_total))
report_total.to_excel(path_resave+'8月末全部提报汇总.xlsx',index=False,encoding='gbk')

# 滚动8月明细时，只取7月的非月度提报加上8月的全部提报
report_total_July = report_total[report_total['提报月份']=='7月'].copy()
report_total_July = report_total_July[report_total_July['项目类型'] != '月度']
report_total_Aug = report_total[report_total['提报月份']=='8月'].copy()
report_total_use = pd.concat([report_total_July,report_total_Aug])
print('使用的提报条数',len(report_total_use))
report_total_use.to_excel(path_resave+'需要滚动8月明细的提报.xlsx',index=False,encoding='gbk')  # 本文件仅用于检查

report_total_dp = report_total_use.drop_duplicates(subset='id',keep='last')

print('去重后使用的提报条数',len(report_total_dp))
# 此处如果有重复，是提报重复

In [ ]:
order = pd.read_csv(path+'8月\\DAY_商用订单明细_8月整.csv',encoding='gbk')
order = order.drop_duplicates(subset=None,keep='last')

order['商品编码'] = order['商品编码'].apply(lambda x:str(x))
order['区域']=order['区域'].apply(lambda x:str(x))

order_F = order[pd.isnull(order['m_contract_number'])].copy()
order_F['id'] = order_F['账号']+','+order_F['商品编码']+','+order_F['区域'].apply(lambda x: x[0:2])+'区'

order_T = order[pd.notnull(order['m_contract_number'])].copy()
order_T['id'] = order_T['m_contract_number']+','+order_T['商品编码']+','+order_T['区域'].apply(lambda x: x[0:2])+'区'

order = pd.concat([order_T,order_F])
order.to_excel(path+'8月\\处理后商用订单明细_8月整.xlsx',index=False, encoding='gbk')

In [ ]:
result = pd.merge(order,report_total_dp[['id','*提报人']],on='id',how='inner')
print(path+"匹配到的数据量",len(result))

path_save = path+'8月\\项目业绩\\'
result.to_excel(path_save+'8月商品池业绩_新.xlsx',index=False,encoding='gbk')

In [ ]:
path_sep = path_save+'区域业绩拆分\\'

result_region = result.drop(columns = ['m_contract_number','id','供应商名称'])

for name, group in result_region.groupby('区域'):
    print(name)
    group.to_excel(path_sep+name+"8月商品池项目业绩.xlsx",encoding = 'gbk',index = False)

In [ ]:
path_sep = path_save+'区域提报拆分\\'

for name, group in report_total.groupby('*区域'):
    print(name)
    group.to_excel(path_sep+name+"7.8月商品池项目提报.xlsx",encoding = 'gbk',index = False)